In [ ]:
# Install dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install py4j

# Set environment variables for Spark
import os
os.environ['SPARK_HOME'] = '/content/spark-3.3.2-bin-hadoop2.7'
os.environ['PYTHONPATH'] = os.path.join(os.environ['SPARK_HOME'], 'python') + ":" + os.path.join(os.environ['SPARK_HOME'], 'python', 'lib', 'py4j-*.zip')

# Initialize findspark
import findspark
findspark.init()

# Start Spark session
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.master("local[*]").appName("dataset").getOrCreate()

^C
ERROR: Operation cancelled by user


Exception: Unable to find py4j in /content/spark-3.3.2-bin-hadoop2.7/python, your SPARK_HOME may not be configured correctly

In [ ]:
#reading the file
df = spark.read.csv('/content/dataset.csv',header=True)

In [ ]:
# View schema
df.printSchema()

# Show a few rows
df.show(5)

root
 |-- Header_Length: string (nullable = true)
 |-- Protocol Type: string (nullable = true)
 |-- Time_To_Live: string (nullable = true)
 |-- Rate: string (nullable = true)
 |-- fin_flag_number: string (nullable = true)
 |-- syn_flag_number: string (nullable = true)
 |-- rst_flag_number: string (nullable = true)
 |-- psh_flag_number: string (nullable = true)
 |-- ack_flag_number: string (nullable = true)
 |-- ece_flag_number: string (nullable = true)
 |-- cwr_flag_number: string (nullable = true)
 |-- ack_count: string (nullable = true)
 |-- syn_count: string (nullable = true)
 |-- fin_count: string (nullable = true)
 |-- rst_count: string (nullable = true)
 |-- HTTP: string (nullable = true)
 |-- HTTPS: string (nullable = true)
 |-- DNS: string (nullable = true)
 |-- Telnet: string (nullable = true)
 |-- SMTP: string (nullable = true)
 |-- SSH: string (nullable = true)
 |-- IRC: string (nullable = true)
 |-- TCP: string (nullable = true)
 |-- UDP: string (nullable = true)
 |-- DHCP:

In [ ]:
# Drop rows with any nulls
df = df.dropna()

# Drop duplicates
df = df.dropDuplicates()

In [ ]:
from pyspark.ml.feature import StringIndexer

# Encode the target column
label_indexer = StringIndexer(inputCol="Label", outputCol="label_index")
df = label_indexer.fit(df).transform(df)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

# Exclude label columns from conversion
columns_to_convert = [c for c in df.columns if c not in ['Label', 'label_index']]

# Cast all other columns to double
for col_name in columns_to_convert:
    df = df.withColumn(col_name, col(col_name).cast(DoubleType()))

# Exclude Label and label_index from features
feature_cols = [col for col in df.columns if col not in ["Label", "label_index"]]

# Assemble all features into one vector column
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_unscaled")
df = assembler.transform(df)

In [ ]:
from pyspark.ml.feature import StandardScaler

# Scale the features
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withMean=True, withStd=True)
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

In [ ]:
# Sample down to 10% to avoid memory issues
sampled_df = df.select("features", "label_index").sample(fraction=0.1, seed=42).toPandas()

In [ ]:
# Convert Spark Vectors to NumPy arrays
X = sampled_df['features'].apply(lambda x: x.toArray()).tolist()
y = sampled_df['label_index']

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

model = XGBClassifier()
model.fit(X, y)

# Evaluate
y_pred = model.predict(X)
print("Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:\n", classification_report(y, y_pred))

Accuracy: 0.8425870101397643
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6651
         1.0       0.78      0.94      0.85      5963
         2.0       1.00      1.00      1.00      4803
         3.0       0.66      0.59      0.62      4742
         4.0       0.71      0.89      0.79      4570
         5.0       1.00      1.00      1.00      4503
         6.0       0.88      0.63      0.74      4326
         7.0       0.53      0.77      0.62      3995
         8.0       0.76      0.49      0.59      3241
         9.0       0.86      0.42      0.57      2685
        10.0       0.97      1.00      0.98      1676
        11.0       1.00      1.00      1.00      1505
        12.0       1.00      1.00      1.00      1303
        13.0       1.00      1.00      1.00      1139
        14.0       1.00      1.00      1.00       669
        15.0       0.99      0.96      0.98       571
        16.0       0.97     